In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
data = pd.read_csv("./movie_data/movies_metadata.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_20836\1386427846.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./movie_data/movies_metadata.csv")


In [48]:
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [49]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [50]:
data = data[['id','genres', 'vote_average', 'vote_count','popularity','title', 'overview']]

In [51]:
# weighted rating (WR) = (v / (v+m)) * R + (m / (v+m)) * C
# R : 개별 영화 평점
# v : 개별 영화에 평점을 투표한 횟수
# m : m 위 안에 들어야 하는 최소 투표 (정하기 나름)
# C : 전체 영화에 대한 평점

In [68]:
tmp_m = data['vote_count'].quantile(0.989)
tmp_m

2048.8530000000246

In [69]:
tmp_data = data.copy().loc[data['vote_count'] >= tmp_m]
tmp_data.shape

(501, 7)

In [70]:
del tmp_data

m = data['vote_count'].quantile(0.99)
data = data.loc[data['vote_count'] >= m]

In [72]:
C = data['vote_average'].mean()

In [73]:
print(m, C)

2183.8199999999924 6.995824175824176


In [74]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']

    return (v / (v+m) * R) + (m / (v+m) * C)

In [75]:
data['score'] = data.apply(weighted_rating, axis=1)

In [77]:
data.head()

,id,genres,vote_average,vote_count,popularity,title,overview,score
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.946943,Toy Story,"Led by Woody, Andy's toys live happily in his ...",7.497627
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.015539,Jumanji,When siblings Judy and Peter discover an encha...,6.945523
31,63,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",7.4,2470.0,12.297305,Twelve Monkeys,"In the year 2035, convict James Cole reluctant...",7.210339
46,807,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...",8.1,5915.0,18.45743,Se7en,Two homicide detectives are on a desperate hun...,7.802263
49,629,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",8.1,3334.0,16.302466,The Usual Suspects,"Held in an L.A. interrogation room, Verbal Kin...",7.662994


In [78]:
data.shape

(455, 8)

In [81]:
data[['genres']].head(2)

,genres
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."


In [82]:
data['geners'] = data['genres'].apply(literal_eval)

In [83]:
data['geners'].head(2)

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
Name: geners, dtype: object

In [84]:
data['geners'] = data['geners'].apply(lambda x : [d['name'] for d in x]).apply(lambda x :"".join(x))

In [85]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,overview,score,geners
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.946943,Toy Story,"Led by Woody, Andy's toys live happily in his ...",7.497627,AnimationComedyFamily
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.015539,Jumanji,When siblings Judy and Peter discover an encha...,6.945523,AdventureFantasyFamily
